<img src="figures/hop-logo-transparent-bunny.png" style="float: right; height: 5em; padding-top: 1em;">

# Hop.SCIMMA Tutorial

*Streaming hub and services for multi-messenger astronomy.*

## About SCIMMA and Hop.SCIMMA

<img src="figures/scimma-hub.png" style="float: right; height: 30em; padding-top: 5em;">

[SCIMMA](https://scimma.org/) is an NSF-funded, multi-institutional project to understand the needs for and develop cyberinfrastructure in support of multi-messenger astronomy (MMA).

One such need is the ability to reliably and with low latency connect MMA experiments, follow-up facilities, and scientists. This has led to development of [Hopskotch](https://scimma.org/projects.html), **a clustered deployment of Apache Kafka strongly integrated with IAM systems common in research space (namely, [InCommon](https://www.incommon.org/) and [CILogon](https://www.cilogon.org/home))**.

SCIMMA also operates [Hop.SCIMMA](https://hop.scimma.org) a deployment of Hopskotch-as-a-service that will be made available for MMA related projects and science. The goals of Hop.SCIMMA include:
1. Provide a hub for MMA alert producers onto which they can publish their discoveries with very small latency (10s of ms), high reliability (no message loss), fine-grained permissions, and ease of use.
1. Provide a hub to which MMA alert consumers (follow-up observatories, groups, individual scientists) can connect to receive MMA alerts, as well as publish enriched or transformed alerts.
1. Provide a reliable, performant, easy to use, platform (PaaS) which projects and individuals can use to exchange messages and connect experiments.

In this short demo, we will show the services available on Hop.SCIMMA, **with a special focus on IAM integration**. We note the interfaces we show will mostly be used by tool builders and power user; most *astronomers* will be using these services through interfaces such as [TOMs](https://lco.global/tomtoolkit/) or systems like [TreasureMap](http://treasuremap.space/). Other demos and talks in this conference will discuss those higher-level integrations.

## Hop.SCIMMA services overview

WARNING: Hop.SCIMMA is currently in "invitation-only beta" phase, but for the duration of this demo we've opened up sign-up to everyone. Because of that, **you should consider you account ephemeral -- it will be removed within 24 hours**.

[Hop.SCIMMA](https://hop.scimma.org) platform currently comprises of:
 1. A deployment of Hopskotch (a cluster of Kafka brokers)
 1. Translators for other alert systems (i.e., a GCN-to-Hopskotch topic bridge)
 1. A deployment of JupyterHub, also integrated with the CILogon IAM system

More services are being planned soon (including ability to make connections to HPC systems, and ML/classification related services).

We've also built [hop-client](https://pypi.org/project/hop-client/), a Python library and command-line utility to make it easy to sending and receive data from Hopskotch. While any Kafka client is able to access the raw streams, `hop-client` comes with automated parsing of messages, listing of topics, querying metadata, and other services that are more cumbersome to access otherwise.

## Demo: Authentication

### General

One of the major benefits of Hop.SCIMMA (and Hopskotch) is the tight integration of IAM system with Apache Kafka. In particular, opening an account to listen on public streams takes less than a minute.

#### Demo:

1. To open an account, go to [Hop.SCIMMA](https://hop.scimma.org/), click 'Sign Up', and follow the prompts to authenticate using your provider of choice (typically, your university or institute). If your university is not part of the [InCommon federation](https://incommon.org/federation/), you can authenticate using other providers such as Google or Facebook.
  * Note how there was no need to enter a password -- all authentication works through existing providers; this makes it easy for the users to sign up and maintain credentials.
  * It also allows us to delegate access control management to groups at your universities (or arbitrary collaborations).

1. After you've signed up you have access to all Hop.SCIMMA services. For example:
  * Going to https://my.hop.scimma.org allows the user to obtain Kafka access credentials, list available topics, and manage their groups and topics (if any) 
  * Going to https://jupyter.scimma.org allows the user to access the Hop.SCIMMA JupyterHub

At this point, you should have accounts and be able to log into JupyterHub and follow along.

If you haven't been able to log in, please let us know via e-mail or Slack and we'll debug offline. Unfortunately, due to the short time available for this demo we won't be able to do much debugging in real time.

### Accessing Kafka

All connections to Hop.SCIMMA Kafka broker -- at kafka.scimma.org -- require authentication.

To support a high-degree of security as well as compatibility with standard Kafka tools, we have integrated token-based authentication into Hopskotch where the tokens to Kafka look like username/password combinations.

To connect to Kafka, each Hop.SCIMMA user first generates one or more access tokens that give them permissions to one or more available topics. To Kafka, these tokens look like usernames and passwords.

#### Demo:
  1. We will generate a token, by going to https://my.hop.scimma.org
  2. We will add it to hop-client's configuration using the `hop auth add` command
  3. We will show how to connect to public streams with `hop-client`

## Demo: Authorization

Hop.SCIMMA is envisioned to be a shared system -- a scalable hub -- allowing users to make data (streams) available at a very granular level, and only to colleagues who are allowed to read them. All access to topics has to be explicitly authorized: the token used to connect to Kafka determines what is accessible.

#### Demo:

We will demonstrate how tokens that were granted different permissions can (or cannot) access different topics.

While we don't have enough time to go into the details, the model implemented in Hopskotch decentralizes the control to individual users and groups.

For example, groups (organizations) can decide to publish a certain set of topics for internal use only. They can also decide to grant access to a subset to organizations or individuals they collaborate with. They can even create "inbox" topics, where the collaborators are allowed to write but not read.

#### Details (for possible offline discussion)
* The security model distinguishes:
  * **users**, who can be members of one or more groups with varying roles within those groups (e.g., "member", or "owner")
  * **groups** (think of them as "organizations") own topics and contain members (users)
    * Topics are prefixed by group names (e.g., `gcn.circular`, or `ligo.public`)
    * A group that owns a topic can give granular permissions to other groups to that topic (e.g. "read", "write", etc.")
  * **topics**, which are owned by groups and private to anyone else by default.

## Demo: Production and Consumption

As shown in the demos above, production and consumption is possible using any Kafka client, given the access token.

## Potential uses within the LSST ecosystem

What can Hop.SCIMMA offer to the broader LSST broker ecosystem?

Technical opportunities:
* The [infrastructure underlying Hop.SCIMMA](https://github.com/scimma) is largely written in Python, and aimed to be reusable and transferrable to other projects. It may be possible to generalize SCIMMA's IAM ("Identity and Access Management") modules for use within other services (and community development).

Operational opportunities:
* Hop.SCIMMA is being considered by LIGO and similar experiments as an end-point to which to transmit machine-readable discoveries. Brokers in the LSST ecosystem can connect to Hop.SCIMMA to receive these messages.
* Hop.SCIMMA coukd be listened to by Rubin observatory itself, for TOO-triggering alerts.


## Future plans

We're currently working on stablizing the Hop.SCIMMA service and ensuring it satisfies requirements of the major MMA data producers.

At the same time, Hopskotch message passing is being integrated with high-level (end-user) tools such as the TOM toolkit and TreasureMap. See demos and talks in this workshop by the Las Cumbres team.

<hr>

<img src="figures/nsf1.jpg" style="float: right; height: 5em">

This project is supported by National Science Foundation grants OAC-1841625, OAC-1934752. Any opinions, findings, conclusions or recommendations expressed in this material are those of the developers and do not necessarily reflect the views of the National Science Foundation.